In [1]:
import requests, os
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from coinapi_rest_v1.restapi import CoinAPIv1

In [2]:
'''Free api key, 100 request per day. If exceed limit too often, will be asked to upgrade'''
api = CoinAPIv1('4C0F25D9-035A-4E89-B38F-B168FFE89E8C')
# api = CoinAPIv1('892571FE-0278-4F02-9A2D-D5CA3B652D45')
# 892571FE-0278-4F02-9A2D-D5CA3B652D45
# 4C0F25D9-035A-4E89-B38F-B168FFE89E8C

In [3]:
# end_date = dt.datetime.combine(dt.date.today(), dt.time.min).isoformat()
# start_date = dt.datetime.combine(dt.date.today() - dt.timedelta(days=2), dt.time.min).isoformat()
# end_date = dt.datetime.combine(dt.date.today() - dt.timedelta(days=1), dt.time.min).isoformat()
# print(start_date, end_date)

In [4]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
        
date_list = [date.isoformat() for date in datetime_range(dt.datetime(2021, 5, 19), dt.datetime(2021, 5, 24), dt.timedelta(days = 1))]

In [5]:
# print(date_list)

In [6]:
# exchanges = api.metadata_list_exchanges()
# print('Exchanges')
# for exchange in exchanges:
#     print('Exchange ID: %s' % exchange['exchange_id'])
#     print('Exchange website: %s' % exchange['website'])
#     print('Exchange name: %s' % exchange['name'])

# periods = api.ohlcv_list_all_periods()

In [7]:
# symbols = api.metadata_list_symbols()
# print('Symbols')
# for symbol in symbols:
#     print('Symbol ID: %s' % symbol['symbol_id'])
#     print('Exchange ID: %s' % symbol['exchange_id'])
#     print('Symbol type: %s' % symbol['symbol_type'])

In [8]:
def appendDFToCSV_void(df, csvFilePath, sep=","):
    if not os.path.isfile(csvFilePath):
        df.to_csv(csvFilePath, mode='w', index=True, sep=sep)
    else: 
        print("File already up-to-date!")

In [9]:
symbol = 'HUOBIPRO_SPOT_ETH_USDT'
period_id = '5MIN'

date_iter = iter(date_list)
for (index, value) in enumerate(date_list[:-1]): 
    ohlcv_list = []
    start_date, end_date = value, date_list[index + 1]
    while start_date != end_date:
        print(start_date)
        ohlcv_historical = api.ohlcv_historical_data(symbol, {'period_id': period_id, 'time_start': start_date, 'time_end': end_date})
        ohlcv_list.append(pd.DataFrame(ohlcv_historical))
        start_date = dt.datetime.strptime(ohlcv_historical[-1]['time_period_end'],"%Y-%m-%dT%H:%M:%S.%f000Z").isoformat()
    
    ohlcv_df = pd.concat(ohlcv_list)
    ohlcv_df.reset_index(drop = True, inplace = True)
    time_period_start = [dt.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%f000Z") for x in ohlcv_df['time_period_start']]
    time_period_end = [dt.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%f000Z") for x in ohlcv_df['time_period_end']]
    final_df = pd.DataFrame(ohlcv_df[['price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']])
    final_df.index = time_period_end
    final_df.columns = ['open', 'high', 'low', 'close', 'volume', 'trade_count']
    final_df.index.names = ['Date']
    
    csvFilePath = r'D:/Database/Crypto data/_coin_api/' + symbol + ' ' + value[:-9] + '.csv'
    appendDFToCSV_void(final_df, csvFilePath)

2021-05-19T00:00:00
2021-05-19T08:20:00
2021-05-19T16:40:00
2021-05-20T00:00:00
2021-05-20T08:20:00
2021-05-20T16:40:00
2021-05-21T00:00:00
2021-05-21T08:20:00
2021-05-21T16:40:00
2021-05-22T00:00:00
2021-05-22T08:20:00
2021-05-22T16:40:00
